# HW 5. Instruction-tuning Llama-2 для выполнения математических операций сложения, умножения, вычитания и деленеия

В данном домашнем задании вам необходимо обучить языковую модель (не более 1B параметров) решать примеры на сложение как можно более длинных чисел.

**Ожидаемый результат**

Необходимо предоставить код, а также технический отчет, содержащий описание метода и используемых данных, оценку качества (точность сложения).

**Пояснения**

1. Можно использовать любые предобученные модели, можно их файнтюнить, обучать с нуля, адаптировать любым другим способом или брать как есть. Главное, чтобы все использованные вами идеи, код или веса моделей были описаны в приложенном отчете со ссылкой на источник.

2. Мы ожидаем, что ваш код принимает на вход два числа (в виде строк их десятичной записи) и выдает ответ в любом человекочитаемом виде. Однако, если ваша модель работает с входом в виде предложения на естественном языке, это тоже нормально, главное, чтобы это было описано в отчете.

3. Можно оценить качество работы алгоритма, посчитав accuracy на случайных множествах чисел разной длины. Если вам кажется более подходящей другая метрика, мы примем ваше решение. Опишите вашу метрику и аргументируйте выбор в отчете.

**Подсказка**

В качестве ориентира можете использовать следующий репозиторий (https://github.com/liutiedong/goat). В нём реализовано сборка датасета для обучения и само обучение.

### Разбаловка

- [2 балла] Сбор датасета для обучения  
- [3 балла] Реализация скрипта модели
- [3 балла] Создание обученной модели с качеством (100% на числах длины < 10)
- [2 баллов] Валидация модели

Для валидации можно использовать и готовую языковую модель без получения баллов за 3 пункт

# План решения

За основу решения возьму указанный репозиторий авторов. В качестве первого шага был сгенерирован датасет в отдельном ноутбуке, посмотреть можно по ссылке. В этом ноутбуке буду использовать уже готовый файл.
В качестве основной модели возьму llama, так как у нее на каждую цифру есть отдельный токен. Вот в этом ноутбуке я тестировала токенайзеры разных моделей, чтобы проверить логику формирвоания токенов для разных разрядов чисел. Как писали авторы в своей статье [https://arxiv.org/abs/2305.14201], так работают токенайзеры Llama 2 и ChatGLM. Я смотрела также модели для русского языка, у них такой возможности нет. Поэтому реализовывать каоткулятор буду на английском языке как и авторы статьи с помощью Instruction tuning для Llama 2.

## Шаг 1. Загрузим библиотеки


In [1]:
#!g2.1
%pip install -q -U einops datasets matplotlib tqdm boto3 git+https://github.com/dask/s3fs

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tokenizers 0.14.1 requires huggingface_hub<0.18,>=0.16.4, but you have huggingface-hub 0.19.3 which is incompatible.
cloud-ml 0.0.1 requires boto3<=1.20.24,>=1.12.31, but you have boto3 1.28.64 which is incompatible.
cloud-ml 0.0.1 requires s3fs<=0.5.2,>=0.4.1, but you have s3fs 2023.10.0+1.g9494426 which is incompatible.
gcsfs 2023.6.0 requires fsspec==2023.6.0, but you have fsspec 2023.10.0 which is incompatible.
spacy 3.5.4 requires pydantic!=1.8,!=1.8.1,<1.11.0,>=1.7.4

In [2]:
#!g2.1
import os
import sys
from typing import List

In [3]:
#!g2.1
%pip install fire

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [4]:
#!g2.1
%pip install transformers

Defaulting to user installation because normal site-packages is not writeable
  Obtaining dependency information for huggingface-hub<1.0,>=0.16.4 from https://files.pythonhosted.org/packages/aa/f3/3fc97336a0e90516901befd4f500f08d691034d387406fdbde85bea827cc/huggingface_hub-0.17.3-py3-none-any.whl.metadata
  Obtaining dependency information for urllib3<1.27,>=1.21.1 from https://files.pythonhosted.org/packages/b0/53/aa91e163dcfd1e5b82d8a890ecf13314e3e149c05270cc644581f77f17fd/urllib3-1.26.18-py2.py3-none-any.whl.metadata
  Using cached urllib3-1.26.18-py2.py3-none-any.whl.metadata (48 kB)
  Using cached charset_normalizer-2.0.12-py3-none-any.whl (39 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 1.8 MB/s eta 0:00:00
Using cached urllib3-1.26.18-py2.py3-none-any.whl (143 kB)
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface

In [5]:
#!g2.1
import fire
import torch
import transformers
from datasets import load_dataset

In [6]:
#!g2.1
from typing import Union

class Prompter(object):
    
    def generate_prompt(
        self,
        instruction: str,
        label: Union[None, str] = None,
    ) -> str:     

        res = f"{instruction}\nAnswer: "
               
        if label:
            res = f"{res}{label}"
         
        return res


    def get_response(self, output: str) -> str:
        return output.split("Answer:")[1].strip().replace("/", "\u00F7").replace("*", "\u00D7")
    
prompter = Prompter()

# Шаг 2. Обработатем датасет

Для этого будем использовать 2 функции, которые будут токенизировать инструкцию. Здесь код без изменений от авторов репозитория.

In [7]:
#!g2.1
def tokenize(prompt, add_eos_token=True):
    result = tokenizer(
        prompt,
        truncation=True,
        max_length=cutoff_len,
        padding=False,
        return_tensors=None,
    )
    if (
        result["input_ids"][-1] != tokenizer.eos_token_id
        and len(result["input_ids"]) < cutoff_len
        and add_eos_token
    ):
        result["input_ids"].append(tokenizer.eos_token_id)
        result["attention_mask"].append(1)

    result["labels"] = result["input_ids"].copy()

    return result

def generate_and_tokenize_prompt(data_point):
    full_prompt = prompter.generate_prompt(
        data_point["instruction"],
        data_point["output"],
    )
    tokenized_full_prompt = tokenize(full_prompt)
    user_prompt = prompter.generate_prompt(
            data_point["instruction"]
        )
    tokenized_user_prompt = tokenize(user_prompt, add_eos_token=False)
    user_prompt_len = len(tokenized_user_prompt["input_ids"])

    tokenized_full_prompt["labels"] = [
        -100
    ] * user_prompt_len + tokenized_full_prompt["labels"][
        user_prompt_len:
    ]  # could be sped up, probably
    return tokenized_full_prompt

In [8]:
#!g2.1
from transformers import AutoTokenizer, AutoModelForCausalLM

In [9]:
#!g2.1
%pip install sentencepiece

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [31]:
#!g2.1
from huggingface_hub import notebook_login
notebook_login()

In [11]:
#!g2.1
number_list = [74815, 7481, 748,74,7]

tokenizer = AutoTokenizer.from_pretrained("huggyllama/llama-7b") # huggyllama/llama-7b
tokenizer.pad_token_id = 0 
tokenizer.padding_side = "left"

for num in number_list:
    print(f"{num}: token {tokenizer(str(num))['input_ids']}")

74815: token [1, 29871, 29955, 29946, 29947, 29896, 29945]
7481: token [1, 29871, 29955, 29946, 29947, 29896]
748: token [1, 29871, 29955, 29946, 29947]
74: token [1, 29871, 29955, 29946]
7: token [1, 29871, 29955]


Так как я запускала блокнот повторно, то не буду в нем еще раз обрабатывать данные, а воспользуюсь уже готовыми, чтобы сэкономить на времени и ресурсах. Код для подготовки файлов ниже и закомментирован.

In [12]:
#!g2.1
from datasets import load_from_disk

val_data = load_from_disk("val_data.hf")
train_data = load_from_disk("train_data.hf")
train_data

Dataset({
    features: ['output', 'answer', 'input', 'instruction', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 1449927
})

In [13]:
#!g2.1

cutoff_len=512

#data = load_dataset("json", data_files='dataset.json')
#train_val = data["train"].train_test_split(test_size=0.00005, shuffle=True, seed=42)

#train_data = (train_val["train"].shuffle().map(generate_and_tokenize_prompt))
#val_data = (train_val["test"].shuffle().map(generate_and_tokenize_prompt))

In [14]:
#!g2.1
for i, l in train_data[0].items():
    print(f'{i}: type {type(l)}')

output: type <class 'str'>
answer: type <class 'str'>
input: type <class 'str'>
instruction: type <class 'str'>
input_ids: type <class 'list'>
attention_mask: type <class 'list'>
labels: type <class 'list'>


In [15]:
#!g2.1
#train_data.save_to_disk("train_data.hf")
#val_data.save_to_disk("val_data.hf")

# Шаг 3. Загрузим модель и настроим LoRa Adapters

Для LoRa воспользуемся параметрами, указанными авторами репозитория. Загрузим модель и настроим для нее адаптеры. Буду использовать библиотеку peft. Посмотрим как поменялось кол-во обучаемых параметров после настройки адаптеров.

In [16]:
#!g2.1
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [17]:
#!g2.1
%pip install -q bitsandbytes accelerate loralib

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2023.6.0 requires fsspec==2023.6.0, but you have fsspec 2023.10.0 which is incompatible.
spacy 3.5.4 requires pydantic!=1.8,!=1.8.1,<1.11.0,>=1.7.4, but you have pydantic 2.5.1 which is incompatible.

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [18]:
#!g2.1
%pip install -q git+https://github.com/huggingface/peft.git

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2023.6.0 requires fsspec==2023.6.0, but you have fsspec 2023.10.0 which is incompatible.
spacy 3.5.4 requires pydantic!=1.8,!=1.8.1,<1.11.0,>=1.7.4, but you have pydantic 2.5.1 which is incompatible.

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [20]:
#!g2.1
model = AutoModelForCausalLM.from_pretrained("baffo32/decapoda-research-llama-7B-hf", load_in_8bit=True,torch_dtype=torch.float16)

Loading checkpoint shards:   0%|          | 0/33 [00:00<?, ?it/s]

In [21]:
#!g2.1
from peft import (
    LoraConfig,
    get_peft_model,
    get_peft_model_state_dict,
    prepare_model_for_int8_training,
    set_peft_model_state_dict,
)

In [22]:
#!g2.1
model = prepare_model_for_int8_training(model)

/home/jupyter/.local/lib/python3.10/site-packages/peft/utils/other.py:136: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(


In [23]:
#!g2.1
lora_r=64
lora_alpha= 64
lora_dropout= 0.05
lora_target_modules = ["q_proj", "v_proj", "k_proj", "o_proj",]

In [24]:
#!g2.1
config = LoraConfig(
        r=lora_r,
        lora_alpha=lora_alpha,
        target_modules=lora_target_modules,
        lora_dropout=lora_dropout,
        bias="none",
        task_type="CAUSAL_LM",
    )

model = get_peft_model(model, config)
model.print_trainable_parameters()

trainable params: 67,108,864 || all params: 6,805,524,480 || trainable%: 0.986093932910223


Здесь видно, как сильно уменьшилось кол-во обучаемых параметров - стало менее 1% от изначальной модели.


In [25]:
#!g2.1
import warnings
warnings.filterwarnings("ignore")

# Шаг 4. Метрика
В качестве метрики буду использовать готовую из библиотеки evaluate exact match. Она будет показывать, процент полного совпадания сгенерированного ответа и референсного ответа. Данную метрику также использовали и авторы, и она очень похожа на accuracy для строк.

Метрику буду использовать после обучения на этапе генерации и подсчитывать буду процент ответов модели, полностью совпадающих с референсным на основе валидационной выборки.

In [26]:
#!g2.1
%pip install evaluate

Defaulting to user installation because normal site-packages is not writeable
  Obtaining dependency information for huggingface-hub>=0.7.0 from https://files.pythonhosted.org/packages/38/f6/06d7489a9f1b2112a640b3272abd43319d0ee625f26efafb350106893c19/huggingface_hub-0.19.3-py3-none-any.whl.metadata
  Using cached huggingface_hub-0.19.3-py3-none-any.whl.metadata (14 kB)
  Obtaining dependency information for urllib3<1.27,>=1.21.1 from https://files.pythonhosted.org/packages/b0/53/aa91e163dcfd1e5b82d8a890ecf13314e3e149c05270cc644581f77f17fd/urllib3-1.26.18-py2.py3-none-any.whl.metadata
  Using cached urllib3-1.26.18-py2.py3-none-any.whl.metadata (48 kB)
  Using cached charset_normalizer-2.0.12-py3-none-any.whl (39 kB)
Using cached huggingface_hub-0.19.3-py3-none-any.whl (311 kB)
Using cached urllib3-1.26.18-py2.py3-none-any.whl (143 kB)
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Attempting uninstall: huggingface-

In [27]:
#!g2.1
import evaluate
metric = evaluate.load("exact_match")

2023-11-16 11:18:43.505058: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [28]:
#!g2.1
%pip install wandb

Defaulting to user installation because normal site-packages is not writeable
  Obtaining dependency information for urllib3<1.27,>=1.21.1 from https://files.pythonhosted.org/packages/b0/53/aa91e163dcfd1e5b82d8a890ecf13314e3e149c05270cc644581f77f17fd/urllib3-1.26.18-py2.py3-none-any.whl.metadata
  Using cached urllib3-1.26.18-py2.py3-none-any.whl.metadata (48 kB)
  Using cached charset_normalizer-2.0.12-py3-none-any.whl (39 kB)
Using cached urllib3-1.26.18-py2.py3-none-any.whl (143 kB)
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2023.6.0 requires fsspec==2023.6.0, but you have fsspec 2023.10.0 which is incompatible.
spacy 3.5.4 requires pydantic!=1.8,!=1.8.1,<1.11.0,>=1.7.4, but you have pydantic 2.5.1 which is incompatible.

[notice] A 

In [29]:
#!g2.1
import wandb
import os

os.environ['WANDB_PROJECT'] = 'Llama'
os.environ['WANDB_LOG_MODEL'] = 'true'

wandb.login(key = 'XXX')

/usr/local/lib/python3.10/dist-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (2.0.7) or chardet (4.0.0)/charset_normalizer (3.3.0) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "
wandb: Currently logged in as: shakhovak1981 (shakhovak). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home/jupyter/.netrc


True

In [30]:
#!g2.1
trainer = transformers.Trainer(
    model=model, 
    train_dataset=train_data,
    eval_dataset=val_data,
    #compute_metrics = compute_metrics,
    args=transformers.TrainingArguments(
        per_device_train_batch_size=8, 
        gradient_accumulation_steps=4,
        warmup_steps=40, 
        num_train_epochs=1,
        max_steps=1000, 
        learning_rate=3e-4, 
        fp16=True,
        logging_steps=500, 
        save_strategy="no",
        optim="adamw_torch",
        evaluation_strategy="steps",
        eval_steps=100,
        output_dir='HW5_Llama2_arithmetic_2',
        report_to="wandb",
        #push_to_hub = True
        
    ),
    data_collator=transformers.DataCollatorForSeq2Seq(
            tokenizer, pad_to_multiple_of=8, return_tensors="pt", padding=True
        ),
)
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!


trainer.train()


wandb: Tracking run with wandb version 0.16.0
wandb: Run data is saved locally in /home/jupyter/work/resources/goat/wandb/run-20231116_111926-1gnoxxel
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run usual-pine-3
wandb: ⭐️ View project at https://wandb.ai/shakhovak/Llama
wandb: 🚀 View run at https://wandb.ai/shakhovak/Llama/runs/1gnoxxel
You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss


Step,Training Loss,Validation Loss


Step,Training Loss,Validation Loss


Step,Training Loss,Validation Loss


...

Step,Training Loss,Validation Loss
100,No log,0.102911
200,No log,0.067008
300,No log,0.046139
400,No log,0.029015
500,0.096900,0.028619
600,0.096900,0.021012
700,0.096900,0.018571
800,0.096900,0.013318
900,0.096900,0.012308


Step,Training Loss,Validation Loss
100,No log,0.102911
200,No log,0.067008
300,No log,0.046139
400,No log,0.029015
500,0.096900,0.028619
600,0.096900,0.021012
700,0.096900,0.018571
800,0.096900,0.013318
900,0.096900,0.012308


Step,Training Loss,Validation Loss
100,No log,0.102911
200,No log,0.067008
300,No log,0.046139
400,No log,0.029015
500,0.096900,0.028619
600,0.096900,0.021012
700,0.096900,0.018571
800,0.096900,0.013318
900,0.096900,0.012308
1000,0.019600,0.011519


Step,Training Loss,Validation Loss
100,No log,0.102911
200,No log,0.067008
300,No log,0.046139
400,No log,0.029015
500,0.096900,0.028619
600,0.096900,0.021012
700,0.096900,0.018571
800,0.096900,0.013318
900,0.096900,0.012308
1000,0.019600,0.011519


TrainOutput(global_step=1000, training_loss=0.05825093078613281, metrics={'train_runtime': 10683.9827, 'train_samples_per_second': 2.995, 'train_steps_per_second': 0.094, 'total_flos': 3.623964990190387e+17, 'train_loss': 0.05825093078613281, 'epoch': 0.02})

У модели есть еще потенциал для дальнейшего обучения, пока остановлюсь на это варианте в целях экономии времени.


In [32]:
#!g2.1
trainer.push_to_hub()

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

adapter_model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/4.16k [00:00<?, ?B/s]

'https://huggingface.co/Shakhovak/HW5_Llama2_arithmetic_2/tree/main/'

In [41]:
#!g2.1
wandb.finish()

# Инференс и проверка качества

Для проверки качества работы модели воспользуюсь функцией для генерации текста. Настройки генерации возьму такие же, как и у авторов репозитория, единственное уменьшу общее кол-во линий генерации до 2-х, так как отслеживать 4 как у авторов требует много времени и ресурсов GPU.


In [33]:
#!g2.1
import torch
device = torch.device ('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [34]:
#!g2.1
from transformers import GenerationConfig

In [35]:
#!g2.1

model.eval()

def evaluat(
    text,
    temperature=0.1,
    top_p=0.75,
    top_k=40,
    num_beams=2,
    max_new_tokens=512,
    stream_output=True,
    **kwargs,
):
    prompt = prompter.generate_prompt(text)
    inputs = tokenizer(prompt, return_tensors="pt")
    input_ids = inputs["input_ids"].to(device)
    generation_config = GenerationConfig(
        temperature=temperature,
        top_p=top_p,
        top_k=top_k,
        num_beams=num_beams,
        **kwargs,
    )

    generate_params = {
        "input_ids": input_ids,
        "generation_config": generation_config,
        "return_dict_in_generate": True,
        "output_scores": True,
        "max_new_tokens": max_new_tokens,
    }

    # Without streaming
    with torch.no_grad():
        generation_output = model.generate(
            input_ids=input_ids,
            generation_config=generation_config,
            return_dict_in_generate=True,
            output_scores=True,
            max_new_tokens=max_new_tokens,
        )
    s = generation_output.sequences[0]
    output = tokenizer.decode(s, skip_special_tokens=True).strip()
    
    return prompter.get_response(output)


In [36]:
#!g2.1
from tqdm import tqdm_notebook
answers = []
for i in tqdm_notebook(val_data):
    answers.append(evaluat(i['instruction']))

  0%|          | 0/73 [00:00<?, ?it/s]

По кадой позиции из валидационной выборке сравню генерацию и референсный ответ и подсчитаю кол-во совпадений. Выведу правильный ответ и ответ, сгенерированный моделью.


In [44]:
#!g2.1
texts_together = []
count = 0
for i in val_data['output']:
  for j in answers:
    
    if val_data['output'].index(i) == answers.index(j):
        j = j.replace('×', '*').replace('÷', '/')
        if metric.compute(predictions=[j], references=[i])['exact_match'] == 1.0:
            count += 1
        texts_together.append((i,j,metric.compute(predictions=[j], references=[i]) ))

display(f'Total % of correct answers: {count/len(texts_together)}')
display('=======================')
for i in texts_together:
    display(i)
    display('===========================')

'Total % of correct answers: 0.7945205479452054'

'======================='

('91 * 94 = 94 * (90 + 1) = 94 * 90 + 94 * 1 = 8460 + 94 = 8554',
 '91 * 94 = 94 * (90 + 1) = 94 * 90 + 94 * 1 = 8460 + 94 = 8554',
 {'exact_match': 1.0})

'==========================='

('8387734 + 294 = 8388028', '8387734 + 294 = 8387763', {'exact_match': 0.0})

...

'==========================='

('27 * 346225 = 346225 * (20 + 7) = 346225 * 20 + 346225 * 7 = 6924500 + 2423575 = 9348075',
 '27 * 346225 = 346225 * (20 + 7) = 346225 * 20 + 346225 * 7 = 6924500 + 2423575 = 9348075',
 {'exact_match': 1.0})

'==========================='

('42736544140 - 35170 * 1000000 = 42736544140 - 35170000000 = 7566544140\n7566544140 - 35170 * 200000 = 7566544140 - 7034000000 = 532544140\n532544140 - 35170 * 10000 = 532544140 - 351700000 = 180844140\n180844140 - 35170 * 5000 = 180844140 - 175850000 = 4994140\n4994140 - 35170 * 100 = 4994140 - 3517000 = 1477140\n1477140 - 35170 * 40 = 1477140 - 1406800 = 70340\n70340 - 35170 * 2 = 70340 - 70340 = 0\nTherefore, 42736544140 / 35170 = 1215142',
 '42736544140 - 35170 * 1000000 = 42736544140 - 35170000000 = 7566544140\n7566544140 - 35170 * 200000 = 7566544140 - 7034000000 = 532544140\n532544140 - 35170 * 10000 = 532544140 - 351700000 = 180844140\n180844140 - 35170 * 5000 = 180844140 - 175850000 = 4994140\n4994140 - 35170 * 100 = 4994140 - 3517000 = 1477140\n1477140 - 35170 * 40 = 1477140 - 1406800 = 70340\n70340 - 35170 * 2 = 70340 - 70340 = 0\nTherefore, 42736544140 / 35170 = 1215142',
 {'exact_match': 1.0})

'==========================='

# Финальные выводы

Результаты не совсем соответсвую требованию (меньше 100%) точность, но как видно из динамики обучения у модели есть еще потенциал для улучшения качества. Интересно, что ошибается модель на 1 или 2 цифры в ответе, в целом правильно поддерживая структуру и формат ответа.
